In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as ss

sns.set_style('white')

%matplotlib inline

In [24]:
loan_df = pd.read_csv('loan.csv')
loan_df.head()

C:\Users\piyus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Number of columns & rows
print("Number of columns: ",loan_df.shape[1])
print("Number of rows: ",loan_df.shape[0])

Number of columns:  74
Number of rows:  887379


In [7]:
#Checking if ids and member_ids are all unique
print("Unique ids:",len(set(loan_df.id)))
print("Unique member ids:",len(set(loan_df.member_id)))

Unique ids: 887379
Unique member ids: 887379


In [8]:
#Column names
loan_df.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
       'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint

In [11]:
#Checking if loan_df has any null values
loan_df.isnull().sum()

id                                  0
member_id                           0
loan_amnt                           0
funded_amnt                         0
funded_amnt_inv                     0
term                                0
int_rate                            0
installment                         0
grade                               0
sub_grade                           0
emp_title                       51457
emp_length                          0
home_ownership                      0
annual_inc                          4
verification_status                 0
issue_d                             0
loan_status                         0
pymnt_plan                          0
url                                 0
desc                           761350
purpose                             0
title                             151
zip_code                            0
addr_state                          0
dti                                 0
delinq_2yrs                        29
earliest_cr_

We will consider rows with less than 10% missing values for filling.

In [12]:
#Columns that need filling missing values
c = 0
print("Column Name: Number of missing values: Percentage of missing values")
for i in range(len(loan_df.columns.values)):
    if ((np.ceil(loan_df[loan_df.columns.values[i]].isnull().sum()*100/887379) > 0) & 
        (np.ceil(loan_df[loan_df.columns.values[i]].isnull().sum()*100/887379) <= 10)):
        print(loan_df.columns.values[i],":",loan_df[loan_df.columns.values[i]].isnull().sum(),":",
              np.ceil(loan_df[loan_df.columns.values[i]].isnull().sum()*100/887379))
        c = c + 1
print("Number of columns with less than 10% Null values: ",c)

Column Name: Number of missing values: Percentage of missing values
emp_title : 51457 : 6.0
annual_inc : 4 : 1.0
title : 151 : 1.0
delinq_2yrs : 29 : 1.0
earliest_cr_line : 29 : 1.0
inq_last_6mths : 29 : 1.0
open_acc : 29 : 1.0
pub_rec : 29 : 1.0
revol_util : 502 : 1.0
total_acc : 29 : 1.0
last_pymnt_d : 17659 : 2.0
last_credit_pull_d : 53 : 1.0
collections_12_mths_ex_med : 145 : 1.0
acc_now_delinq : 29 : 1.0
tot_coll_amt : 70276 : 8.0
tot_cur_bal : 70276 : 8.0
total_rev_hi_lim : 70276 : 8.0
Number of columns with less than 10% Null values:  17


We have 17 columns that require filling missing values.

In [26]:
#Check columns with more than 10% missing values so that they can be dropped from loan_df
c = 0
loan_df_filtered = loan_df
print("Column Name: Number of missing values: Percentage of missing values")
for i in range(len(loan_df.columns.values)):
    if (np.ceil(loan_df[loan_df.columns.values[i]].isnull().sum()*100/887379) > 10):
        loan_df_filtered = loan_df_filtered.drop(loan_df.columns.values[i],axis=1)
        print(loan_df.columns.values[i],":",loan_df[loan_df.columns.values[i]].isnull().sum(),":",
              np.ceil(loan_df[loan_df.columns.values[i]].isnull().sum()*100/887379))
        c = c + 1
print("Number of columns with more than 10% Null values: ",c)

Column Name: Number of missing values: Percentage of missing values
desc : 761350 : 86.0
mths_since_last_delinq : 454312 : 52.0
mths_since_last_record : 750326 : 85.0
next_pymnt_d : 252971 : 29.0
mths_since_last_major_derog : 665676 : 76.0
annual_inc_joint : 886868 : 100.0
dti_joint : 886870 : 100.0
verification_status_joint : 886868 : 100.0
open_acc_6m : 866007 : 98.0
open_il_6m : 866007 : 98.0
open_il_12m : 866007 : 98.0
open_il_24m : 866007 : 98.0
mths_since_rcnt_il : 866569 : 98.0
total_bal_il : 866007 : 98.0
il_util : 868762 : 98.0
open_rv_12m : 866007 : 98.0
open_rv_24m : 866007 : 98.0
max_bal_bc : 866007 : 98.0
all_util : 866007 : 98.0
inq_fi : 866007 : 98.0
total_cu_tl : 866007 : 98.0
inq_last_12m : 866007 : 98.0
Number of columns with more than 10% Null values:  22


We created loan_df_filtered which is a copy of loan_df but without the above 22 columns.

In [27]:
#Number of columns & rows in loan_df_filtered
print("Number of columns: ",loan_df_filtered.shape[1])
print("Number of rows: ",loan_df_filtered.shape[0])

Number of columns:  52
Number of rows:  887379


In [15]:
#Column names in loan_df_filtered
loan_df_filtered.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'url', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim'], dtype=object)

We will drop the columns that would not (theoretically) affect interest rate.

In [28]:
arr = ['id','member_id','pymnt_plan','url','dti','revol_bal','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','emp_title'
         ,'total_pymnt_inv','total_rec_prncp','total_rec_int','recoveries','collection_recovery_fee','last_pymnt_amnt'
         ,'policy_code','application_type','title','earliest_cr_line','revol_util','last_pymnt_d'
        ,'last_credit_pull_d','collections_12_mths_ex_med','acc_now_delinq','tot_coll_amt','total_rev_hi_lim','total_rec_late_fee']

In [29]:
loan_df_filtered = loan_df_filtered.drop(arr,axis=1)

In [30]:
#Number of columns
print("Number of columns: ",loan_df_filtered.shape[1])

Number of columns:  23


In [31]:
#Column names in loan_df_filtered
loan_df_filtered.columns.values

array(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'purpose', 'zip_code', 'addr_state', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'tot_cur_bal'], dtype=object)

We will now categorize columns as categorical and continuous. So we will check their value counts.

In [33]:
print("Column Name: Number of unique values")
for i in range(len(loan_df_filtered.columns.values)):
    print(loan_df_filtered.columns.values[i],":",len(loan_df_filtered[loan_df_filtered.columns.values[i]].unique()))

Column Name: Number of unique values
loan_amnt : 1372
funded_amnt : 1372
funded_amnt_inv : 9856
term : 2
int_rate : 542
installment : 68711
grade : 7
sub_grade : 35
emp_length : 12
home_ownership : 6
annual_inc : 49385
verification_status : 3
issue_d : 103
loan_status : 10
purpose : 14
zip_code : 935
addr_state : 51
delinq_2yrs : 30
inq_last_6mths : 29
open_acc : 78
pub_rec : 33
total_acc : 136
tot_cur_bal : 327343


We will consider columns containing less than 150 unique values to be categorical features.

In [37]:
for i in range(len(loan_df_filtered.columns.values)):
    if(len(loan_df_filtered[loan_df_filtered.columns.values[i]].unique()) <= 150):
        print(loan_df_filtered.columns.values[i], ":")
        print(loan_df_filtered[loan_df_filtered.columns.values[i]].value_counts(),"\n")

term :
 36 months    621125
 60 months    266254
Name: term, dtype: int64 

grade :
B    254535
C    245860
A    148202
D    139542
E     70705
F     23046
G      5489
Name: grade, dtype: int64 

sub_grade :
B3    56323
B4    55626
C1    53387
C2    52236
C3    50161
C4    48857
B5    48833
B2    48781
B1    44972
A5    44816
C5    41219
D1    36238
A4    34531
D2    29803
D3    26554
D4    25558
A3    23457
A1    22913
A2    22485
D5    21389
E1    18268
E2    17004
E3    14134
E4    11724
E5     9575
F1     7218
F2     5392
F3     4433
F4     3409
F5     2594
G1     1871
G2     1398
G3      981
G4      663
G5      576
Name: sub_grade, dtype: int64 

emp_length :
10+ years    291569
2 years       78870
< 1 year      70605
3 years       70026
1 year        57095
5 years       55704
4 years       52529
n/a           44825
7 years       44594
8 years       43955
6 years       42950
9 years       34657
Name: emp_length, dtype: int64 

home_ownership :
MORTGAGE    443557
RENT        356117

In [39]:
print("Column Name: Number of missing values")
for i in range(len(loan_df_filtered.columns.values)):
    if(loan_df_filtered[loan_df_filtered.columns.values[i]].isnull().sum()>0):
        print(loan_df_filtered.columns.values[i],":",loan_df_filtered[loan_df_filtered.columns.values[i]].isnull().sum())

Column Name: Number of missing values
annual_inc : 4
delinq_2yrs : 29
inq_last_6mths : 29
open_acc : 29
pub_rec : 29
total_acc : 29
tot_cur_bal : 70276


We have to fill missing values in the above mentioned 7 features.

In [40]:
#We will first save the dataframe as a .csv file
loan_df_filtered.to_csv('loan_filtered.csv')

In [41]:
#Read loan_df
loan_df = pd.read_csv('loan_filtered.csv', usecols=range(1,24) ,encoding = "latin-1")
loan_df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,loan_status,purpose,zip_code,addr_state,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,tot_cur_bal
0,5000.0,5000.0,4975.000000,36 months,10.65,162.87,B,B2,10+ years,RENT,...,Fully Paid,credit_card,860xx,AZ,0.0,1.0,3.0,0.0,9.0,NaN
1,2500.0,2500.0,2500.000000,60 months,15.27,59.83,C,C4,< 1 year,RENT,...,Charged Off,car,309xx,GA,0.0,5.0,3.0,0.0,4.0,NaN
2,2400.0,2400.0,2400.000000,36 months,15.96,84.33,C,C5,10+ years,RENT,...,Fully Paid,small_business,606xx,IL,0.0,2.0,2.0,0.0,10.0,NaN
3,10000.0,10000.0,10000.000000,36 months,13.49,339.31,C,C1,10+ years,RENT,...,Fully Paid,other,917xx,CA,0.0,1.0,10.0,0.0,37.0,NaN
4,3000.0,3000.0,3000.000000,60 months,12.69,67.79,B,B5,1 year,RENT,...,Current,other,972xx,OR,0.0,0.0,15.0,0.0,38.0,NaN
5,5000.0,5000.0,5000.000000,36 months,7.90,156.46,A,A4,3 years,RENT,...,Fully Paid,wedding,852xx,AZ,0.0,3.0,9.0,0.0,12.0,NaN
6,7000.0,7000.0,7000.000000,60 months,15.96,170.08,C,C5,8 years,RENT,...,Current,debt_consolidation,280xx,NC,0.0,1.0,7.0,0.0,11.0,NaN
7,3000.0,3000.0,3000.000000,36 months,18.64,109.43,E,E1,9 years,RENT,...,Fully Paid,car,900xx,CA,0.0,2.0,4.0,0.0,4.0,NaN
8,5600.0,5600.0,5600.000000,60 months,21.28,152.39,F,F2,4 years,OWN,...,Charged Off,small_business,958xx,CA,0.0,2.0,11.0,0.0,13.0,NaN
9,5375.0,5375.0,5350.000000,60 months,12.69,121.45,B,B5,< 1 year,RENT,...,Charged Off,other,774xx,TX,0.0,0.0,2.0,0.0,3.0,NaN


In [42]:
#columns x rows
print(loan_df.shape[1],loan_df.shape[0])

23 887379


In [43]:
#column names
loan_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'issue_d', 'loan_status',
       'purpose', 'zip_code', 'addr_state', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'total_acc', 'tot_cur_bal'],
      dtype='object')

In [44]:
#Annual income had 4 nan values. So filled by highest count - 60000.00
loan_df['annual_inc'] = loan_df['annual_inc'].fillna(60000.00)

In [45]:
#Check unique values of total current balance
print(np.size(pd.unique(loan_df['tot_cur_bal'])))
#Check number of nan Values
print(loan_df['tot_cur_bal'].isnull().values.sum())

327343
70276


In [46]:
#Check correlation between features to fill tot_cur_bal
#Using annual_inc since corr between annual_inc and tot_cur_bal is highest = 0.418332 
loan_df.corr(method='pearson')

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,tot_cur_bal
loan_amnt,1.000000,0.999263,0.997115,0.145023,0.944977,0.332698,-0.000159,-0.034168,0.198906,-0.081177,0.222639,0.328782
funded_amnt,0.999263,1.000000,0.998025,0.145160,0.946005,0.332467,0.000151,-0.034711,0.199387,-0.080836,0.222575,0.328781
funded_amnt_inv,0.997115,0.998025,1.000000,0.145205,0.943632,0.331420,0.000648,-0.038880,0.200004,-0.079790,0.222793,0.328804
int_rate,0.145023,0.145160,0.145205,1.000000,0.133075,-0.072784,0.055178,0.227650,-0.010381,0.052156,-0.038618,-0.091408
installment,0.944977,0.946005,0.943632,0.133075,1.000000,0.326182,0.007876,-0.003623,0.183868,-0.069967,0.200409,0.297898
annual_inc,0.332698,0.332467,0.331420,-0.072784,0.326182,1.000000,0.047470,0.035193,0.138392,-0.008341,0.187335,0.418332
delinq_2yrs,-0.000159,0.000151,0.000648,0.055178,0.007876,0.047470,1.000000,0.021375,0.051189,-0.011004,0.122713,0.064762
inq_last_6mths,-0.034168,-0.034711,-0.038880,0.227650,-0.003623,0.035193,0.021375,1.000000,0.111235,0.056870,0.135584,0.034338
open_acc,0.198906,0.199387,0.200004,-0.010381,0.183868,0.138392,0.051189,0.111235,1.000000,-0.025301,0.695075,0.244134
pub_rec,-0.081177,-0.080836,-0.079790,0.052156,-0.069967,-0.008341,-0.011004,0.056870,-0.025301,1.000000,0.012472,-0.075814


In [47]:
#70k null values in tot_cur_bal (8% data)
#Use bivariate regression to fill values of tot_cur_bal using annual_inc
#1. Take tot_cur_bal and annual_inc in a separate df
#2. Divide null and not null in two sets - test and training
#3. Use training to train model and fill values to test
#4. Fill values in original dataset

#1. Take tot_cur_bal and annual_inc in a separate df
bal_inc_df = pd.DataFrame(loan_df,columns = ['tot_cur_bal','annual_inc'])
print(bal_inc_df.head())

   tot_cur_bal  annual_inc
0          NaN     24000.0
1          NaN     30000.0
2          NaN     12252.0
3          NaN     49200.0
4          NaN     80000.0


In [48]:
#2. Divide null and not null in two sets - test and training

bal_inc_df_test = bal_inc_df[bal_inc_df.isnull().any(axis=1)]
bal_inc_df_train = bal_inc_df.dropna(axis=0)
bal_inc_df_train = bal_inc_df_train.reset_index()
bal_inc_df_test = bal_inc_df_test.reset_index()

In [49]:
bal_inc_df_test

,index,tot_cur_bal,annual_inc
0,0,NaN,24000.00
1,1,NaN,30000.00
2,2,NaN,12252.00
3,3,NaN,49200.00
4,4,NaN,80000.00
5,5,NaN,36000.00
6,6,NaN,47004.00
7,7,NaN,48000.00
8,8,NaN,40000.00
9,9,NaN,15000.00


In [50]:
bal_inc_df_train

,index,tot_cur_bal,annual_inc
0,42535,114834.0,55000.0
1,42536,14123.0,26000.0
2,42537,267646.0,105000.0
3,42538,13605.0,40000.0
4,42539,272492.0,63000.0
5,42540,327264.0,130000.0
6,42541,17672.0,88000.0
7,42542,5759.0,28000.0
8,42543,39143.0,102000.0
9,42544,19530.0,25000.0


In [51]:
bal_inc_df_pred = bal_inc_df_test['tot_cur_bal']

In [52]:
X_inc = np.array(bal_inc_df_train['annual_inc'])
X_bal = np.array(bal_inc_df_train['tot_cur_bal'])
Y_bal = np.array(bal_inc_df_test['annual_inc'])

In [53]:
#3. Use training to train model and fill values to test
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_inc.reshape(817103,1),X_bal.reshape(817103,1))
df = reg.predict(Y_bal.reshape(70276,1))

In [54]:
bal_inc_df_test['tot_cur_bal'] = df

In [55]:
bal_inc_df_test

,index,tot_cur_bal,annual_inc
0,0,88692.076096,24000.00
1,1,94600.112503,30000.00
2,2,77124.140811,12252.00
3,3,113505.829005,49200.00
4,4,143833.749228,80000.00
5,5,100508.148910,36000.00
6,6,111343.487680,47004.00
7,7,112324.221724,48000.00
8,8,104446.839848,40000.00
9,9,79830.021485,15000.00


In [ ]:
#Replace index values from bal_inc_df_test to indexes of loan_df['tot_cur_bal']
for i in range(bal_inc_df_test.shape[0]):
    loan_df['tot_cur_bal'][bal_inc_df_test['index'][i]] = bal_inc_df_test['tot_cur_bal'][i]

C:\Users\piyus\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [ ]:
#Checking if values have been filled
loan_df['tot_cur_bal'][230655]

In [168]:
#Confirming No null values in total current balance
print(loan_df['tot_cur_bal'].isnull().values.sum())

0


In [170]:
#Checking for any other Columns with null values
for i in range(len(loan_df.columns.values)):
    if (loan_df[loan_df.columns.values[i]].isnull().sum() > 0):
        print(loan_df.columns.values[i],":",loan_df[loan_df.columns.values[i]].isnull().sum(),"\n")

inq_last_6mths : 29 

open_acc : 29 

pub_rec : 29 

total_acc : 29 



In [171]:
loan_df[loan_df.isnull().any(axis=1)]

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,...,loan_status,purpose,addr_state,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,total_rec_late_fee,tot_cur_bal
42449,79912,5000.0,5000.0,3025.0,36 months,7.43,155.38,A,A2,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,NY,0.0,NaN,NaN,NaN,NaN,0.000000,124140.294538
42450,79906,7000.0,7000.0,3450.0,36 months,7.75,218.55,A,A3,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,NY,0.0,NaN,NaN,NaN,NaN,0.000000,124140.294538
42459,70473,4350.0,4350.0,825.0,36 months,8.07,136.45,A,A4,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,MA,0.0,NaN,NaN,NaN,NaN,0.000000,183220.658609
42472,72194,3900.0,3900.0,2025.0,36 months,9.33,124.62,B,B3,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,NY,0.0,NaN,NaN,NaN,NaN,0.000000,161557.858449
42480,79878,6700.0,6700.0,6700.0,36 months,7.75,209.18,A,A3,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,NY,0.0,NaN,NaN,NaN,NaN,0.000000,124140.294538
42483,70651,3200.0,3200.0,2000.0,36 months,9.96,103.20,B,B5,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,MA,0.0,NaN,NaN,NaN,NaN,0.000000,212760.840644
42494,72816,7200.0,5075.0,350.0,36 months,10.28,164.42,C,C1,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,NY,0.0,NaN,NaN,NaN,NaN,0.000000,158603.840246
42509,70879,1900.0,1900.0,900.0,36 months,9.64,61.00,B,B4,1 year,...,Does not meet the credit policy. Status:Fully ...,debt_consolidation,FL,0.0,NaN,NaN,NaN,NaN,0.000000,163527.203918
42514,85923,1200.0,1200.0,500.0,36 months,9.01,38.17,B,B2,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,TX,0.0,NaN,NaN,NaN,NaN,0.000000,100508.148910
42515,85802,5000.0,5000.0,375.0,36 months,11.22,164.23,C,C4,< 1 year,...,Does not meet the credit policy. Status:Fully ...,other,NJ,0.0,NaN,NaN,NaN,NaN,29.994759,92630.767034


Found exactly 29 rows with missing values. We can drop them.

In [173]:
loan_df = loan_df.dropna(axis=0)
loan_df = loan_df.reset_index(drop = True)
loan_df

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,...,loan_status,purpose,addr_state,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,total_rec_late_fee,tot_cur_bal
0,1296599,5000.0,5000.0,4975.000000,36 months,10.65,162.87,B,B2,10+ years,...,Fully Paid,credit_card,AZ,0.0,1.0,3.0,0.0,9.0,0.00,88692.076096
1,1314167,2500.0,2500.0,2500.000000,60 months,15.27,59.83,C,C4,< 1 year,...,Charged Off,car,GA,0.0,5.0,3.0,0.0,4.0,0.00,94600.112503
2,1313524,2400.0,2400.0,2400.000000,36 months,15.96,84.33,C,C5,10+ years,...,Fully Paid,small_business,IL,0.0,2.0,2.0,0.0,10.0,0.00,77124.140811
3,1277178,10000.0,10000.0,10000.000000,36 months,13.49,339.31,C,C1,10+ years,...,Fully Paid,other,CA,0.0,1.0,10.0,0.0,37.0,16.97,113505.829005
4,1311748,3000.0,3000.0,3000.000000,60 months,12.69,67.79,B,B5,1 year,...,Current,other,OR,0.0,0.0,15.0,0.0,38.0,0.00,143833.749228
5,1311441,5000.0,5000.0,5000.000000,36 months,7.90,156.46,A,A4,3 years,...,Fully Paid,wedding,AZ,0.0,3.0,9.0,0.0,12.0,0.00,100508.148910
6,1304742,7000.0,7000.0,7000.000000,60 months,15.96,170.08,C,C5,8 years,...,Current,debt_consolidation,NC,0.0,1.0,7.0,0.0,11.0,0.00,111343.487680
7,1288686,3000.0,3000.0,3000.000000,36 months,18.64,109.43,E,E1,9 years,...,Fully Paid,car,CA,0.0,2.0,4.0,0.0,4.0,0.00,112324.221724
8,1306957,5600.0,5600.0,5600.000000,60 months,21.28,152.39,F,F2,4 years,...,Charged Off,small_business,CA,0.0,2.0,11.0,0.0,13.0,0.00,104446.839848
9,1306721,5375.0,5375.0,5350.000000,60 months,12.69,121.45,B,B5,< 1 year,...,Charged Off,other,TX,0.0,0.0,2.0,0.0,3.0,0.00,79830.021485


In [191]:
#Confirming No null values in the entire dataframe (loan_df)
print(loan_df.isnull().values.sum())

0


In [7]:
loan_df.columns

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'purpose', 'zip_code', 'addr_state',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc',
       'total_rec_late_fee', 'tot_cur_bal'],
      dtype='object')

In [174]:
#Verify correlation between between annual_inc and tot_cur_bal if it's similar after filling na values = 0.429346
loan_df.corr(method='pearson')

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,total_rec_late_fee,tot_cur_bal
member_id,1.000000,0.064404,0.066697,0.072236,-0.142254,0.025977,0.034145,0.049904,-0.127257,0.088838,0.080788,0.031692,-0.076362,0.012176
loan_amnt,0.064404,1.000000,0.999263,0.997118,0.144993,0.944974,0.332707,-0.000159,-0.034168,0.198906,-0.081177,0.222639,0.031466,0.323810
funded_amnt,0.066697,0.999263,1.000000,0.998028,0.145130,0.946002,0.332476,0.000151,-0.034711,0.199387,-0.080836,0.222575,0.031146,0.323829
funded_amnt_inv,0.072236,0.997118,0.998028,1.000000,0.145167,0.943631,0.331435,0.000648,-0.038880,0.200004,-0.079790,0.222793,0.028061,0.323054
int_rate,-0.142254,0.144993,0.145130,0.145167,1.000000,0.133044,-0.072786,0.055178,0.227650,-0.010381,0.052156,-0.038618,0.057207,-0.085427
installment,0.025977,0.944974,0.946002,0.943631,0.133044,1.000000,0.326192,0.007876,-0.003623,0.183868,-0.069967,0.200409,0.041884,0.294097
annual_inc,0.034145,0.332707,0.332476,0.331435,-0.072786,0.326192,1.000000,0.047470,0.035193,0.138392,-0.008341,0.187335,0.013544,0.429346
delinq_2yrs,0.049904,-0.000159,0.000151,0.000648,0.055178,0.007876,0.047470,1.000000,0.021375,0.051189,-0.011004,0.122713,0.017432,0.064673
inq_last_6mths,-0.127257,-0.034168,-0.034711,-0.038880,0.227650,-0.003623,0.035193,0.021375,1.000000,0.111235,0.056870,0.135584,0.031637,0.032033
open_acc,0.088838,0.198906,0.199387,0.200004,-0.010381,0.183868,0.138392,0.051189,0.111235,1.000000,-0.025301,0.695075,-0.009606,0.239931


Keep the quantitative features and the qualitative ones eligible for one-hot encoding. 
Drop features with large unique values, since they ain't useful for one-hot encoding.

keep - loan_amnt, funded_amnt, term, int_rate, installment, grade, sub_grade, emp_length<br>
keep - home_ownership, annual_inc, verification_status, purpose, addr_state, inq_last_6mths<br>
keep - pub_rec, total_acc, total_rec_late_fee <br>
drop - issue_d (103 unique values), zip_code(935 unique values)


Categorical features with non-numeric categories: term, grade, sub_grade, emp_length, home_ownership, verification_status, loan_status, purpose, addr_state <br>
14 other features - Check for categorical features with numerical values (dummies separately)<br>
 -> open_acc, delinq_2yrs, inq_last_6mths, pub_rec <br>
Drop columns with numeric categorical values - encode them and append to original data frame.

In [72]:
#Drop features
loan_df = loan_df.drop(['loan_status','funded_amnt_inv','issue_d','zip_code'],axis=1)

In [9]:
#Save file in CSV
loan_df.to_csv("data/loan.csv")

In [10]:
#Checkpoint 1
loan_df = pd.read_csv("data/loan.csv", usecols=range(1,20) ,encoding = "latin-1")
loan_df.head()

,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,tot_cur_bal
0,5000.0,5000.0,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,credit_card,AZ,0.0,1.0,3.0,0.0,9.0,88692.076096
1,2500.0,2500.0,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,car,GA,0.0,5.0,3.0,0.0,4.0,94600.112503
2,2400.0,2400.0,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,small_business,IL,0.0,2.0,2.0,0.0,10.0,77124.140811
3,10000.0,10000.0,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,other,CA,0.0,1.0,10.0,0.0,37.0,113505.829005
4,3000.0,3000.0,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,other,OR,0.0,0.0,15.0,0.0,38.0,143833.749228


In [11]:
print(loan_df.columns)
print(len(loan_df.columns))

Index(['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'purpose', 'addr_state', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'tot_cur_bal'],
      dtype='object')
19


In [12]:
#Make copy for later use
loan_df_all = loan_df
loan_df_all_cp = loan_df_all

In [13]:
#Create dummies for features with categorical non-numeric values
loan_df_all = pd.get_dummies(loan_df_all, columns = ['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
                                                 'verification_status', 'purpose', 'addr_state'])
loan_df_all_shape = loan_df_all.shape
print("Number of columns: "+ str(loan_df_all_shape[1]))
print("Number of rows: "+ str(loan_df_all_shape[0]))
print(loan_df_all.columns.values)

Number of columns: 141
Number of rows: 887350
['loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'annual_inc'
 'delinq_2yrs' 'inq_last_6mths' 'open_acc' 'pub_rec' 'total_acc'
 'tot_cur_bal' 'term_ 36 months' 'term_ 60 months' 'grade_A' 'grade_B'
 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G' 'sub_grade_A1'
 'sub_grade_A2' 'sub_grade_A3' 'sub_grade_A4' 'sub_grade_A5' 'sub_grade_B1'
 'sub_grade_B2' 'sub_grade_B3' 'sub_grade_B4' 'sub_grade_B5' 'sub_grade_C1'
 'sub_grade_C2' 'sub_grade_C3' 'sub_grade_C4' 'sub_grade_C5' 'sub_grade_D1'
 'sub_grade_D2' 'sub_grade_D3' 'sub_grade_D4' 'sub_grade_D5' 'sub_grade_E1'
 'sub_grade_E2' 'sub_grade_E3' 'sub_grade_E4' 'sub_grade_E5' 'sub_grade_F1'
 'sub_grade_F2' 'sub_grade_F3' 'sub_grade_F4' 'sub_grade_F5' 'sub_grade_G1'
 'sub_grade_G2' 'sub_grade_G3' 'sub_grade_G4' 'sub_grade_G5'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years

In [14]:
#Remove one of each categorical values too avoid dummy variable trap
#So removing term_ 60 months, grade_G, sub_grade_G5, emp_length_n/a, home_ownership_ANY, verification_status_Not Verified,
#purpose_other, addr_state_WY
loan_df_all = loan_df_all.drop(['term_ 60 months', 'grade_G', 'sub_grade_G5', 'emp_length_n/a', 'home_ownership_ANY', 
                                'verification_status_Not Verified', 'purpose_other', 'addr_state_WY'],axis=1)
print(len(loan_df_all.columns))
print(loan_df_all.columns.values)

133
['loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'annual_inc'
 'delinq_2yrs' 'inq_last_6mths' 'open_acc' 'pub_rec' 'total_acc'
 'tot_cur_bal' 'term_ 36 months' 'grade_A' 'grade_B' 'grade_C' 'grade_D'
 'grade_E' 'grade_F' 'sub_grade_A1' 'sub_grade_A2' 'sub_grade_A3'
 'sub_grade_A4' 'sub_grade_A5' 'sub_grade_B1' 'sub_grade_B2' 'sub_grade_B3'
 'sub_grade_B4' 'sub_grade_B5' 'sub_grade_C1' 'sub_grade_C2' 'sub_grade_C3'
 'sub_grade_C4' 'sub_grade_C5' 'sub_grade_D1' 'sub_grade_D2' 'sub_grade_D3'
 'sub_grade_D4' 'sub_grade_D5' 'sub_grade_E1' 'sub_grade_E2' 'sub_grade_E3'
 'sub_grade_E4' 'sub_grade_E5' 'sub_grade_F1' 'sub_grade_F2' 'sub_grade_F3'
 'sub_grade_F4' 'sub_grade_F5' 'sub_grade_G1' 'sub_grade_G2' 'sub_grade_G3'
 'sub_grade_G4' 'emp_length_1 year' 'emp_length_10+ years'
 'emp_length_2 years' 'emp_length_3 years' 'emp_length_4 years'
 'emp_length_5 years' 'emp_length_6 years' 'emp_length_7 years'
 'emp_length_8 years' 'emp_length_9 years' 'emp_length_< 1 year'
 'home_ownership_MO

In [15]:
#Drop features with numeric categorical data - open_acc, delinq_2yrs, inq_last_6mths, pub_rec 
loan_df_all = loan_df_all.drop(['open_acc', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec'],axis=1)

In [16]:
loan_df_all.shape[1]

129

Create dummies of dropped variables and append to loan_df_all - open_acc, delinq_2yrs, inq_last_6mths, pub_rec

In [17]:
#inq_last_6mths getdummies
inq_df = loan_df_all_cp['inq_last_6mths']
inq_df_all = pd.get_dummies(inq_df, prefix = "inq", prefix_sep='_')
print("Number of columns: "+ str(inq_df_all.shape[1]))
print("Number of rows: "+ str(inq_df_all.shape[0]))
inq_df_all.columns

Number of columns: 28
Number of rows: 887350


Index(['inq_0.0', 'inq_1.0', 'inq_2.0', 'inq_3.0', 'inq_4.0', 'inq_5.0',
       'inq_6.0', 'inq_7.0', 'inq_8.0', 'inq_9.0', 'inq_10.0', 'inq_11.0',
       'inq_12.0', 'inq_13.0', 'inq_14.0', 'inq_15.0', 'inq_16.0', 'inq_17.0',
       'inq_18.0', 'inq_19.0', 'inq_20.0', 'inq_24.0', 'inq_25.0', 'inq_27.0',
       'inq_28.0', 'inq_31.0', 'inq_32.0', 'inq_33.0'],
      dtype='object')

In [18]:
#delinq_2yrs getdummies
delinq_df = loan_df_all_cp['delinq_2yrs']
delinq_df_all = pd.get_dummies(delinq_df, prefix = "delinq", prefix_sep='_')
print("Number of columns: "+ str(delinq_df_all.shape[1]))
print("Number of rows: "+ str(delinq_df_all.shape[0]))
delinq_df_all.columns

Number of columns: 29
Number of rows: 887350


Index(['delinq_0.0', 'delinq_1.0', 'delinq_2.0', 'delinq_3.0', 'delinq_4.0',
       'delinq_5.0', 'delinq_6.0', 'delinq_7.0', 'delinq_8.0', 'delinq_9.0',
       'delinq_10.0', 'delinq_11.0', 'delinq_12.0', 'delinq_13.0',
       'delinq_14.0', 'delinq_15.0', 'delinq_16.0', 'delinq_17.0',
       'delinq_18.0', 'delinq_19.0', 'delinq_20.0', 'delinq_21.0',
       'delinq_22.0', 'delinq_24.0', 'delinq_26.0', 'delinq_27.0',
       'delinq_29.0', 'delinq_30.0', 'delinq_39.0'],
      dtype='object')

In [19]:
#open_acc getdummies
open_acc_df = loan_df_all_cp['open_acc']
open_acc_df_all = pd.get_dummies(open_acc_df, prefix = "open_acc", prefix_sep='_')
print("Number of columns: "+ str(open_acc_df_all.shape[1]))
print("Number of rows: "+ str(open_acc_df_all.shape[0]))
open_acc_df_all.columns

Number of columns: 77
Number of rows: 887350


Index(['open_acc_0.0', 'open_acc_1.0', 'open_acc_2.0', 'open_acc_3.0',
       'open_acc_4.0', 'open_acc_5.0', 'open_acc_6.0', 'open_acc_7.0',
       'open_acc_8.0', 'open_acc_9.0', 'open_acc_10.0', 'open_acc_11.0',
       'open_acc_12.0', 'open_acc_13.0', 'open_acc_14.0', 'open_acc_15.0',
       'open_acc_16.0', 'open_acc_17.0', 'open_acc_18.0', 'open_acc_19.0',
       'open_acc_20.0', 'open_acc_21.0', 'open_acc_22.0', 'open_acc_23.0',
       'open_acc_24.0', 'open_acc_25.0', 'open_acc_26.0', 'open_acc_27.0',
       'open_acc_28.0', 'open_acc_29.0', 'open_acc_30.0', 'open_acc_31.0',
       'open_acc_32.0', 'open_acc_33.0', 'open_acc_34.0', 'open_acc_35.0',
       'open_acc_36.0', 'open_acc_37.0', 'open_acc_38.0', 'open_acc_39.0',
       'open_acc_40.0', 'open_acc_41.0', 'open_acc_42.0', 'open_acc_43.0',
       'open_acc_44.0', 'open_acc_45.0', 'open_acc_46.0', 'open_acc_47.0',
       'open_acc_48.0', 'open_acc_49.0', 'open_acc_50.0', 'open_acc_51.0',
       'open_acc_52.0', 'open_acc_5

In [20]:
#pub_rec getdummies
rec_df = loan_df_all_cp['pub_rec']
rec_df_all = pd.get_dummies(rec_df, prefix = "rec", prefix_sep='_')
print("Number of columns: "+ str(rec_df_all.shape[1]))
print("Number of rows: "+ str(rec_df_all.shape[0]))
rec_df_all.columns

Number of columns: 32
Number of rows: 887350


Index(['rec_0.0', 'rec_1.0', 'rec_2.0', 'rec_3.0', 'rec_4.0', 'rec_5.0',
       'rec_6.0', 'rec_7.0', 'rec_8.0', 'rec_9.0', 'rec_10.0', 'rec_11.0',
       'rec_12.0', 'rec_13.0', 'rec_14.0', 'rec_15.0', 'rec_16.0', 'rec_17.0',
       'rec_18.0', 'rec_19.0', 'rec_20.0', 'rec_21.0', 'rec_22.0', 'rec_23.0',
       'rec_26.0', 'rec_28.0', 'rec_34.0', 'rec_40.0', 'rec_49.0', 'rec_54.0',
       'rec_63.0', 'rec_86.0'],
      dtype='object')

In [21]:
#Remove one feature from each of inq_df_all,delinq_df_all,open_acc_df_all,rec_df_all to avoid dummy trap
#Removing rec_86.0, open_acc_90.0, delinq_39.0, inq_33.0 
inq_df_all = inq_df_all.drop(['inq_33.0'],axis=1)
delinq_df_all = delinq_df_all.drop(['delinq_39.0'],axis=1)
open_acc_df_all = open_acc_df_all.drop(['open_acc_90.0'],axis=1)
rec_df_all = rec_df_all.drop(['rec_86.0'],axis=1)

Concatinate rec_df_all, open_acc_df_all, delinq_df_all & inq_df_all to loan_df_all

In [22]:
#Combine all
df_loan = pd.concat([loan_df_all, rec_df_all, open_acc_df_all, delinq_df_all, inq_df_all], axis = 1)

In [23]:
print("Number of columns: "+ str(df_loan.shape[1]))
print("Number of rows: "+ str(df_loan.shape[0]))

Number of columns: 291
Number of rows: 887350


In [24]:
#Save file in CSV
df_loan.to_csv("data/loan_dummies.csv")

In [25]:
#Checkpoint 2
df_loan = pd.read_csv("data/loan_dummies.csv", usecols=range(1,292) ,encoding = "latin-1")
df_loan

,loan_amnt,funded_amnt,int_rate,installment,annual_inc,total_acc,tot_cur_bal,term_ 36 months,grade_A,grade_B,...,inq_17.0,inq_18.0,inq_19.0,inq_20.0,inq_24.0,inq_25.0,inq_27.0,inq_28.0,inq_31.0,inq_32.0
0,5000.0,5000.0,10.65,162.87,24000.00,9.0,88692.076096,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2500.0,2500.0,15.27,59.83,30000.00,4.0,94600.112503,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2400.0,2400.0,15.96,84.33,12252.00,10.0,77124.140811,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000.0,10000.0,13.49,339.31,49200.00,37.0,113505.829005,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3000.0,3000.0,12.69,67.79,80000.00,38.0,143833.749228,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,5000.0,5000.0,7.90,156.46,36000.00,12.0,100508.148910,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,7000.0,7000.0,15.96,170.08,47004.00,11.0,111343.487680,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3000.0,3000.0,18.64,109.43,48000.00,4.0,112324.221724,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5600.0,5600.0,21.28,152.39,40000.00,13.0,104446.839848,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,5375.0,5375.0,12.69,121.45,15000.00,3.0,79830.021485,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#save int_rate as loan_int_rate
loan_int_rate = df_loan['int_rate']

In [27]:
#drop int_rate since its the dependent variable
loan_data = df_loan.drop(['int_rate'],axis=1)

In [28]:
#Without Interest Rate 290 features
loan_data.shape[1]

290

In [29]:
#1. Feature Selection with int-rate and variance threshold 0.8 
# - including int-rate simply to check whether it also has variance more than 0.8
from sklearn.feature_selection import VarianceThreshold
#labels = df_loan.columns
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df_loan)
labels = sel.get_support(indices=True)
print(labels)
print(df_loan[labels].columns)
print(len(df_loan[labels].columns))
df_loan[labels].head()

[  0   1   2   3   4   5   6   7   9  10  49  59  63  64  65  67  68 264
 265]
Index(['loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'annual_inc',
       'total_acc', 'tot_cur_bal', 'term_ 36 months', 'grade_B', 'grade_C',
       'emp_length_10+ years', 'home_ownership_MORTGAGE',
       'home_ownership_RENT', 'verification_status_Source Verified',
       'verification_status_Verified', 'purpose_credit_card',
       'purpose_debt_consolidation', 'inq_0.0', 'inq_1.0'],
      dtype='object')
19


,loan_amnt,funded_amnt,int_rate,installment,annual_inc,total_acc,tot_cur_bal,term_ 36 months,grade_B,grade_C,emp_length_10+ years,home_ownership_MORTGAGE,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,inq_0.0,inq_1.0
0,5000.0,5000.0,10.65,162.87,24000.0,9.0,88692.076096,1,1,0,1,0,1,0,1,1,0,0,1
1,2500.0,2500.0,15.27,59.83,30000.0,4.0,94600.112503,0,0,1,0,0,1,1,0,0,0,0,0
2,2400.0,2400.0,15.96,84.33,12252.0,10.0,77124.140811,1,0,1,1,0,1,0,0,0,0,0,0
3,10000.0,10000.0,13.49,339.31,49200.0,37.0,113505.829005,1,0,1,1,0,1,1,0,0,0,0,1
4,3000.0,3000.0,12.69,67.79,80000.0,38.0,143833.749228,0,1,0,0,0,1,1,0,0,0,1,0


In [30]:
#2. Feature Selection without int-rate and variance threshold 0.8
from sklearn.feature_selection import VarianceThreshold
#labels = df_loan.columns
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(loan_data)
labels = sel.get_support(indices=True)
print(labels)
print(loan_data[labels].columns)
print(len(loan_data[labels].columns))
loan_data[labels].head()

[  0   1   2   3   4   5   6   8   9  48  58  62  63  64  66  67 263 264]
Index(['loan_amnt', 'funded_amnt', 'installment', 'annual_inc', 'total_acc',
       'tot_cur_bal', 'term_ 36 months', 'grade_B', 'grade_C',
       'emp_length_10+ years', 'home_ownership_MORTGAGE',
       'home_ownership_RENT', 'verification_status_Source Verified',
       'verification_status_Verified', 'purpose_credit_card',
       'purpose_debt_consolidation', 'inq_0.0', 'inq_1.0'],
      dtype='object')
18


,loan_amnt,funded_amnt,installment,annual_inc,total_acc,tot_cur_bal,term_ 36 months,grade_B,grade_C,emp_length_10+ years,home_ownership_MORTGAGE,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,inq_0.0,inq_1.0
0,5000.0,5000.0,162.87,24000.0,9.0,88692.076096,1,1,0,1,0,1,0,1,1,0,0,1
1,2500.0,2500.0,59.83,30000.0,4.0,94600.112503,0,0,1,0,0,1,1,0,0,0,0,0
2,2400.0,2400.0,84.33,12252.0,10.0,77124.140811,1,0,1,1,0,1,0,0,0,0,0,0
3,10000.0,10000.0,339.31,49200.0,37.0,113505.829005,1,0,1,1,0,1,1,0,0,0,0,1
4,3000.0,3000.0,67.79,80000.0,38.0,143833.749228,0,1,0,0,0,1,1,0,0,0,1,0


In [31]:
#Save file in CSV
loan_data[labels].to_csv("data/loan_select.csv")

In [32]:
#Checkpoint 3
df_loan = pd.read_csv("data/loan_dummies.csv", usecols=range(1,293) ,encoding = "latin-1")
loan_sel = pd.read_csv("data/loan_select.csv", usecols=range(1,19) ,encoding = "latin-1")
loan_int_rate = pd.DataFrame(df_loan.int_rate)

The data to build a loan interest model is now ready.